# User Rep Scores
- Goal: output a csv for each user in the datahunt, with each row corresponding to a single answer
- only looking at the parent questions for now


In [1]:
import numpy as np
import pandas as pd
# import scipy as sp
# from scipy import stats

The file path of the datahunt is the only thing that needs to be inputted. This code assumes that the datahunt file name will contain the type of task in some form.

In [2]:
# fill this in
datahunt_file = "testing-format/BETA_Language-2020-05-20T0110-DataHunt.csv"

In [3]:
# Kai's convergence answer csv
answers_df = pd.read_csv("convergence/Answer_Consensus.csv")

# read csv into df
df = pd.read_csv(datahunt_file)


In [4]:
# assigns a task type based on the name of the inputted datahunt csv
if "language" in datahunt_file.lower():
    task_type = "language"
elif "probability" in datahunt_file.lower():
    task_type = "probability"
elif "reasoning" in datahunt_file.lower():
    task_type = "reasoning"
else:
    task_type = "evidence"
    
print("Task type is " + task_type)

Task type is language


In [5]:
def get_question_base_label(row):
    if "A" not in row['Question Label']:
        return row['Question Label']
    else:
        base_end_index = row['Question Label'].index("A") - 1
        return row['Question Label'][:base_end_index]

In [6]:
# uses the function above to add the row "Question Base Label" to the answers df, so there is a uniform 
# format of question labels (select all questions were in a different form)
answers_df["Question Base Label"] = answers_df.apply(get_question_base_label, axis=1)
answers_df

,Unnamed: 0,Task File,Article Number,Question Label,Answer Label,Question Base Label
0,0,Evidence,1712,T1.Q11,T1.Q11.A2,T1.Q11
1,1,Evidence,1712,T1.Q13,T1.Q13.A5,T1.Q13
2,2,Evidence,1712,T1.Q14,T1.Q14.A7,T1.Q14
3,3,Evidence,1712,T1.Q3,0,T1.Q3
4,4,Evidence,1712,T1.Q4,T1.Q4.A4,T1.Q4
...,...,...,...,...,...,...
313,313,Probability,100026,T1.Q5,T1.Q5.A1,T1.Q5
314,314,Probability,100026,T1.Q6,T1.Q6.A1,T1.Q6
315,315,Reasoning,1712,T1.Q10,T1.Q10.A8,T1.Q10
316,316,Reasoning,1712,T1.Q3,T1.Q3.A7,T1.Q3


In [7]:
# array of questions that are "parent" questions
language_parents = ["T1.Q1", "T1.Q12"]
probability_parents = ["T1.Q1", "T1.Q5", "T1.Q6", "T1.Q11"]
reasoning_parents = ["T1.Q1"]
evidence_parents = ["T1.Q1", "T1.Q12"]

# corresponding list of question types, language_parents[n] maps to language_parent_types[n]
language_parents_types = ["select_all", "ordinal"]
probability_parents_types = ["ordinal", "ordinal", "select_one", "ordinal"]
reasoning_parents_types = ["select_all"]
evidence_parents_types = ["select_one", "ordinal"]

# corresponding list of max number of problems, used for scoring ordinal questions
language_num_answers = [13, 4]
probability_num_answers = [3, 3, 3, 4]
reasoning_num_answers = [6]
evidence_num_answers = [3, 4]

In [8]:
# cuts down answers_df to the rows with the relevant task type
if task_type == "language":
    task_answers_df = answers_df.loc[answers_df["Task File"] == "Language"]
    parent_questions = language_parents
    num_answers = language_num_answers
elif task_type == "probability":
    task_answers_df = answers_df.loc[answers_df["Task File"] == "Probability"]
    parent_questions = probability_parents
    num_answers = probability_num_answers
elif task_type == "reasoning":
    task_answers_df = answers_df.loc[answers_df["Task File"] == "Reasoning"]
    parent_questions = reasoning_parents
    num_answers = reasoning_num_answers
elif task_type == "evidence":
    task_answers_df = answers_df.loc[answers_df["Task File"] == "Evidence"]
    parent_questions = evidence_parents
    num_answers = evidence_num_answers
else:
    print("Invalid task type")


In [9]:
# set parent_df to the relevant columns of the parent questions
df = df[["contributor_uuid", "quiz_task_uuid", "article_number", "question_label", 
         "answer_label", "answer_text", "quiz_taskrun_uuid", "finish_time"]]

# creates a dictionary mapping the task type to an list of which questions are parent questions
task_parents = {"language": language_parents, "probability": probability_parents,
                "reasoning": reasoning_parents, "evidence": evidence_parents}
task_parents_types = {"language": language_parents_types, "probability": probability_parents_types,
                "reasoning": reasoning_parents_types, "evidence": evidence_parents_types}


parent_df = df.loc[df['question_label'].isin(task_parents[task_type])]

In [10]:
# adds a column with the question_type of the question_label
question_types = []
for i in parent_df['question_label']:
    question_type = task_parents_types[task_type][task_parents[task_type].index(i)]
    question_types.append(question_type)
        
parent_df.insert(2, "question_type", question_types, True)


In [11]:
# task_answers_df.loc[task_answers_df["Question Base Label"] == 'T1.Q1']
task_answers_df
task_answers_df.loc[task_answers_df["Question Base Label"] == "T1.Q12"].loc[task_answers_df["Article Number"] == 1712]

,Unnamed: 0,Task File,Article Number,Question Label,Answer Label,Question Base Label
85,85,Language,1712,T1.Q12,T1.Q12.A1,T1.Q12


Functions for each question type that take in a row from parent_df and output a row with the relevant information (including score) for that user.

Ordinal questions are scored by how far the answer was from the converged answer, where the penalty for each "one" they are off by is 1 / half the total number of questions.

Select-one questions are given scores of 1 if they select the converged answer, 0 otherwise.

Select-all questions are scored by finding the proportion of "correct" selections they selected, where a "correct" selection is 0 or 1, depending on if the converged set of selections for that question contained a given selection. Thus, false positives and false negatives are included in that calculation.

In [12]:
def ordinal_question(row):
    article_number = row["article_number"]
    question_label = row["question_label"]
    answer_label = int(row["answer_label"][-1])
    correct_answer = task_answers_df.loc[task_answers_df["Article Number"] == article_number]
    correct_answer = correct_answer.loc[correct_answer["Question Base Label"] == question_label]
    correct_answer = int(correct_answer["Answer Label"].iloc[0][-1])
    
    question_num_answers = num_answers[parent_questions.index(row["question_label"])]
    penalty = 1 / (question_num_answers / 2)
    
    return 1 - abs(answer_label - correct_answer) * penalty
    

In [13]:
def select_one_question(row):
    article_number = row["article_number"]
    question_label = row["question_label"]
    answer_label = row["answer_label"]
    correct_answer = task_answers_df.loc[task_answers_df["Article Number"] == article_number]
    correct_answer = correct_answer.loc[correct_answer["Question Base Label"] == question_label]
    correct_answer = correct_answer["Answer Label"].iloc[0]
    
    if answer_label == correct_answer:
        return 1
    else:
        return 0
    

In [14]:
def select_all_question(row):
    user_id = row["contributor_uuid"]
    response_id = row["quiz_taskrun_uuid"]
    
    if user_dfs[user_id].loc[user_dfs[user_id]["quiz_taskrun_uuid"] == response_id].empty:
        article_number = row["article_number"]
        question_label = row["question_label"]
        answer_label = row["answer_label"]

        question_user_df = parent_df.loc[parent_df["quiz_taskrun_uuid"] == response_id]
        question_answers_df = task_answers_df.loc[(task_answers_df["Article Number"] == article_number) &\
                                                 (task_answers_df["Question Base Label"] == question_label)]

        correct_question_counter = 0
        total_question_counter = 0
        
        for index, row in question_answers_df.iterrows():
            if question_user_df.loc[question_user_df["answer_label"] == row["Question Label"]].empty:
                if row["Answer Label"] == "0":
                    correct_question_counter += 1
            else:
                if row["Answer Label"] == "1":
                    correct_question_counter += 1
            total_question_counter += 1

        overall_question_score = correct_question_counter / total_question_counter

        return overall_question_score
    
    return -1
      

In [15]:
# parent_df

The below cell creates a library that maps contributor_uuids to a df where each row contains a question they answered and the score they received for that question

In [16]:
user_dfs = {}

for index, row in parent_df.iterrows():
    if row['contributor_uuid'] not in user_dfs:
        user_dfs[row['contributor_uuid']] = pd.DataFrame(columns=['quiz_task_uuid', 'quiz_taskrun_uuid', 'question_label', 'question_type', 'answer_score', 'time_stamp'])
    if row['question_type'] == "select_all":
        score = select_all_question(row)
    elif row['question_type'] == "select_one":
        score = select_one_question
    else:
        score = ordinal_question(row)
    if score != -1:
        user_dfs[row['contributor_uuid']] = user_dfs[row['contributor_uuid']].append({'quiz_task_uuid': row['quiz_task_uuid'], 'quiz_taskrun_uuid': row['quiz_taskrun_uuid'], 'question_label': row['question_label'], 'question_type': row['question_type'], 'answer_score': score, 'time_stamp': row['finish_time']}, ignore_index=True)
        
    

In [17]:
# example of df for one user
user_dfs["f9143626-bfe0-4e69-b652-6d1525ab4eb0"]

,quiz_task_uuid,quiz_taskrun_uuid,question_label,question_type,answer_score,time_stamp
0,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,834a1f2e-ec75-48b3-976c-f8afc38601fb,T1.Q1,select_all,0.833333,2019-09-04 22:50:32.477936
1,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,834a1f2e-ec75-48b3-976c-f8afc38601fb,T1.Q12,ordinal,1.000000,2019-09-04 22:50:32.477936
2,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,188881d0-4d23-491e-9989-2fc0e1b1c849,T1.Q1,select_all,0.750000,2019-09-04 23:58:28.876952
3,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,188881d0-4d23-491e-9989-2fc0e1b1c849,T1.Q12,ordinal,1.000000,2019-09-04 23:58:28.876952
4,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,963a6810-1cec-4585-ba41-c7453465879e,T1.Q1,select_all,0.833333,2019-09-05 01:42:09.906186
5,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,963a6810-1cec-4585-ba41-c7453465879e,T1.Q12,ordinal,1.000000,2019-09-05 01:42:09.906186
6,f65e3f32-7347-42b3-a6b2-39bc62e226c2,6d196fca-ed52-43c4-9818-a920f9fa789c,T1.Q1,select_all,0.750000,2019-09-04 23:59:12.997157
7,f65e3f32-7347-42b3-a6b2-39bc62e226c2,6d196fca-ed52-43c4-9818-a920f9fa789c,T1.Q12,ordinal,0.500000,2019-09-04 23:59:12.997157
8,f65e3f32-7347-42b3-a6b2-39bc62e226c2,4184dcca-4395-4df4-9d24-8c6b5dc0d6c2,T1.Q1,select_all,0.666667,2019-09-05 01:42:24.534878
9,f65e3f32-7347-42b3-a6b2-39bc62e226c2,4184dcca-4395-4df4-9d24-8c6b5dc0d6c2,T1.Q12,ordinal,1.000000,2019-09-05 01:42:24.534878
